In [ ]:
import nest_asyncio
from pprint import pprint

nest_asyncio.apply()

In [ ]:
%load_ext autoreload
%autoreload 2

from lewidi_lib import (
    assign_cols_perf_metrics,
    enable_logging,
    join_dataset_and_preds,
    load_dataset,
    load_listof_parquets,
    load_preds,
    preds_file,
    process_rdf,
)

enable_logging()


dataset = "MP"
split = "train"
task = "soft-label"
template_id = "60"
file = preds_file(
    dataset=dataset,
    split=split,
    template=template_id,
    model_id="Qwen/Qwen3-32B",
    run_name="1000ex_10loops",
)
rdf = load_listof_parquets([file])
rdf = process_rdf(
    rdf, discard_invalid_pred=True, task=task, response_contains_steps=True
)
ddf = load_dataset(dataset=dataset, split=split, task=task)
joint_df = join_dataset_and_preds(ddf, rdf)
joint_df = assign_cols_perf_metrics(joint_df, task=task)

In [ ]:
row = joint_df.iloc[230]
pprint(row["text"])
print("pred: ", row["pred"])
print("target: ", row["target"])
print("ws_loss: ", row["ws_loss"])

In [ ]:
print(row["reasoning"])

In [ ]:
from judge_lib import make_judge_template


template = make_judge_template(
    judge_template_id=100, dataset=dataset, pred_template_id=template_id
)

In [ ]:
prompt = template.make_prompt(data=row)
# print(propmt)

In [ ]:
from llmlib.openai.openai_completion import (
    OpenAIModel,
    config_for_openrouter,
    LlmReq,
    Message,
)

model = OpenAIModel(
    model_id="deepseek/deepseek-chat-v3-0324",
    **config_for_openrouter(),
    max_new_tokens=10_000,
)

In [ ]:
batch = [LlmReq(convo=[Message.from_prompt(prompt)])]
# batch = [LlmReq(convo=[Message.from_prompt("Do irish swear a lot?")])]
gen = model.complete_batchof_reqs(batch=batch)
res = list(gen)

In [ ]:
print(res[0]["response"])